In [1]:
import vcf
import pandas as pd

def checkgt(record,s):
    try:
        return(record.genotype(s)[GTFIELD])
    except:
        return None
    
def GetGT(gt):
    #print gt
    if gt is None: return None
    if str(gt) == "." or str(gt) == "./.": return None #Missing genotype control
    if "|" in gt: return map(int, gt.split("|"))
    elif "/" in gt: return map(int, gt.split("/"))
    else: return [int(gt), int(gt)] # haploid

def NormalizeGT(genotypes, NORM=True, PRINT_ALLELES=False, MINCOUNT=1, COUNTFILTERS=False):
    def getsum(x):
        if x is None: return None
        else: return sum(x)
    def getstr(x):
        if x is None: return "NA,NA"
        else: return ",".join(map(str, x))
    if PRINT_ALLELES: return map(getstr, genotypes)
    gtsum = map(getsum, genotypes)
    
    if COUNTFILTERS:
        return gtsum # just counting filters, don't waste time to actually normalize
    if not NORM: return gtsum
    gtmean = np.mean([item for item in gtsum if item is not None])
    gtsd = math.sqrt(np.var([item for item in gtsum if item is not None]))
    if gtsd == 0: return None
    def norm(gt, m, s):
        if gt is None: return None
        else: return (gt-m)*1.0/s
    gtnorm = [norm(item, gtmean, gtsd) for item in gtsum]
    return gtnorm    
    
    
VCFFILE='/storage/szfeupe/Runs/650GTEx_estr/Filter_Merged_STRs_All_Samples_New.vcf.gz'
vcf_reader = vcf.Reader(open(VCFFILE, "rb"))
SAMPLES = [item for item in vcf_reader.samples]

In [2]:
def PruneRareGT(gtsum, MINCOUNT):
    """
    For genotypes that occur less than MINCOUNT times, set those to None
    """
    gts = set([item for item in gtsum if item is not None])
    for gt in gts:
        if gtsum.count(gt) < MINCOUNT:
            gtsum = map(lambda x: SetNone(x, gt), gtsum)
    return gtsum

GTFIELD='GB'
NORM = True
COUNTFILTERS = False
PRINT_ALLELES = False
FILTER=True
i=0
for record in vcf_reader:
    if (FILTER==True) and (record.FILTER != [] ): 
        continue
    else:
        genotypes = [checkgt(record,s) for s in SAMPLES]
        genotypes = [s for s in genotypes if str(s) !='None']
        W = [GetGT(checkgt(record,s)) for s in SAMPLES]
        W = [s for s in W if s is not None]
        #NG = NormalizeGT(W, NORM=NORM, PRINT_ALLELES=PRINT_ALLELES, MINCOUNT=20, COUNTFILTERS=COUNTFILTERS)
        print (len(genotypes), '  ',len(W),'  ',record.ID,'  ', genotypes[:5], record.call_rate)
        i+=1
        if i>10:
            break
    

649    649    STR_271    ['-3|-3', '-3|-3', '-3|-3', '-3|-3', '-3|-3'] 0.0
645    645    STR_278    ['0|-4', '-4|-4', '-4|0', '-4|0', '-4|0'] 0.0
647    647    STR_292    ['0|-5', '-5|-5', '0|-5', '0|0', '0|-5'] 0.0
621    621    STR_294    ['0|0', '0|0', '0|0', '0|1', '0|0'] 0.0
595    595    STR_303    ['0|-1', '-1|-1', '-1|-1', '-1|-1', '0|-1'] 0.0
578    578    STR_308    ['0|0', '0|1', '1|1', '0|0', '0|1'] 0.0
603    603    STR_357    ['0|0', '0|0', '0|0', '0|0', '0|0'] 0.0
595    595    STR_358    ['0|0', '0|0', '0|0', '0|0', '0|0'] 0.0
641    641    STR_360    ['-2|-2', '-2|-2', '0|-2', '0|0', '0|-2'] 0.0
605    605    STR_449    ['0|0', '0|0', '0|0', '0|1', '0|0'] 0.0
639    639    STR_456    ['0|0', '0|0', '0|0', '0|3', '0|0'] 0.0015384615384615385


In [4]:
print(record.aaf, record.call_rate) #but there are more than 80% calls 639 out of 650

[0.0] 0.0015384615384615385
